In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_spending_by_mcg = '''SELECT time_period_value, mcg, spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value != '2025Q1'  
and merchant_channel = 'Online'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and mcg != 'All'
and mcc != 'All'
GROUP BY mcg, time_period_value, spend
ORDER BY time_period_value, spend DESC'''

yearly_mcg_df = bq.read_bq_table_sql(client, UK_spending_by_mcg)

yearly_mcg_df['year'] = yearly_mcg_df['time_period_value'].str[:4]

yearly_mcg_df.head()

In [ ]:
yearly_mcg_df.to_csv('yearly_mcg_df.csv')
df = pd.read_csv('yearly_mcg_df.csv')
latest_mcc_df = df[df['year'] == 2024][['mcg', 'spend']]
latest_mcc_df.to_csv('latest_mcg_df.csv', index=False)

In [ ]:
total_spend_per_mcc = latest_mcc_df.groupby('mcg')['spend'].sum()
print(total_spend_per_mcc)

In [ ]:
# does not use mcc = all due to unknown data. 
total_spend_per_mcc_perc = (total_spend_per_mcc / total_spend_per_mcc.sum()) * 100
print(total_spend_per_mcc_perc)

In [ ]:
# Plot the bar chart
ax = total_spend_per_mcc_perc.plot(kind='bar', color='skyblue')

# Customize the plot
plt.ylabel('Percentage of Online Spend')
plt.title('Top 10 MCC by Percentage of Total Online Spend, 2024')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()



In [ ]:
print(top_10_mcc)